In [1]:
import matplotlib.pyplot as plt, numpy as np, scipy.constants as ct
# import electrode
from electrode import (System, PolygonPixelElectrode, euler_matrix,
    PointPixelElectrode, PotentialObjective,
    PatternRangeConstraint, shaped, utils)
from scipy.signal import argrelextrema
import math
from scipy.optimize import minimize

In [3]:
# n electrodes per row (10 default)
# number of rows (2 default)
# width of electrode
# height of electrode
# spacing between electrodes
# tuple of coordinates to start each row
    # top row x-coordinate (bottom left point)
    # top row y-coordinate (bottom left point)

#...
# bottom row x-coordinate (bottom left point)
# bottom row y-coordinate (bottom left point)
def trap(spacing,
        gnd_width, gnd_height, gnd_start_pos,
        dc_width, dc_height, dc_start_pos,
        rf_width, rf_height, rf_start_pos,
        dc_mid_width, dc_mid_height, dc_mid_start_pos,
        n=9, r=2):

    electrodes = []

    #add dc electrodes
    #define effective size, without dielectric gaps
    w_eff = dc_width + spacing
    h_eff = dc_height + spacing
    #start with top row
    row = 0
    for sp in dc_start_pos:
        xp_start = sp[0] - spacing/2
        yp = sp[1] - spacing/2
        for electrode in range(n):
            # e.g. electrode 2 would be leftmost in top row
            # e.g. electrode 12 would be leftmost in bottom row
            electrode_id = str(row * (n+1) + electrode + 2)
            x_shift = electrode * w_eff
            xp = xp_start + x_shift
            # print(xp)
            electrodes.append(
                (electrode_id, [[
                    (xp, yp),
                    (xp + w_eff, yp),
                    (xp + w_eff, yp + h_eff),
                    (xp, yp + h_eff)
                ]])
            )
        row += 1
    dc_mid_w_eff = dc_mid_width + spacing
    dc_mid_h_eff = dc_mid_height + spacing
    for i in range (2):
        xp = dc_mid_start_pos[i][0] - spacing/2
        yp = dc_mid_start_pos[i][1] - spacing/2
        dc_mid_id = str(10*i + 1)
        electrodes.append(
            (dc_mid_id, [[
                (xp, yp),
                (xp + dc_mid_w_eff, yp),
                (xp + dc_mid_w_eff, yp + dc_mid_h_eff),
                (xp, yp + dc_mid_h_eff)
            ]])
        )
    
    # add rf electrodes
    rf_w_eff = rf_width + spacing
    rf_h_eff = rf_height + spacing
    rf_electrodes = []
    for i in range (2):
        xp = rf_start_pos[i][0] - spacing/2
        yp = rf_start_pos[i][1] - spacing/2
        # rf_e_id = str('r'+i+1) #use this for unique rf electrode id
        rf_electrodes.append([
            (xp, yp),
            (xp + rf_w_eff, yp),
            (xp + rf_w_eff, yp + rf_h_eff),
            (xp, yp + rf_h_eff)
        ])
        rf_e_id = str('r')
    electrodes.append(
        (rf_e_id, rf_electrodes)
    )
    
    # add gnd electrodes
    #define effective size, without dielectric gaps
    gnd_w_eff = gnd_width + spacing
    gnd_h_eff = gnd_height + spacing
    gnd_electrodes = []
    for sp in gnd_start_pos:
        xp = sp[0] - spacing/2
        yp = sp[1] - spacing/2
        for gnd in range(n):
            gnd_id = 'gnd'
            gnd_electrodes.append([
                (xp, yp),
                (xp + gnd_w_eff, yp),
                (xp + gnd_w_eff, yp + gnd_h_eff),
                (xp, yp + gnd_h_eff)
            ])
    electrodes.append(
        (gnd_id, gnd_electrodes)
    )
    print(electrodes)
    s = System([PolygonPixelElectrode(name=n, paths=map(np.array, p))
                for n, p in electrodes])
    # apply rf potential to electrode
    # s['r'].rf = 1.
    return s

In [4]:
spacing = 5.
n = 9
dc_width = 120.
# dc_height = 1000.
gnd_width = 300.
gnd_height = 1000.
dc_height = 1000.
dc_mid_height = 20.
# dc_mid_width = n * dc_width + (n-1) * spacing
dc_mid_width = n * dc_width + (n+1) * spacing + 2 * gnd_width
rf_height = 70.
# rf_width = n * dc_width + (n-1) * spacing
rf_width = n * dc_width + (n+1) * spacing + 2 * gnd_width

leftmost_electrode_x = -(4.5*dc_width + 4*spacing)
rightmost_electrode_x = (4.5*dc_width + 4*spacing)

gnd_start_pos = [(leftmost_electrode_x-gnd_width-spacing, (2.5*spacing + dc_mid_height + rf_height)),
                (leftmost_electrode_x-gnd_width-spacing, -(2.5*spacing + dc_mid_height + rf_height + dc_height)),
                (rightmost_electrode_x+spacing, (2.5*spacing + dc_mid_height + rf_height)),
                (rightmost_electrode_x+spacing, -(2.5*spacing + dc_mid_height + rf_height + dc_height))
                ]

dc_start_pos = [(leftmost_electrode_x, (2.5*spacing + dc_mid_height + rf_height)),
                (leftmost_electrode_x, -(2.5*spacing + dc_mid_height + rf_height + dc_height))]
rf_start_pos = [(leftmost_electrode_x-gnd_width-spacing, (1.5*spacing + dc_mid_height)),
                (leftmost_electrode_x-gnd_width-spacing, -(1.5*spacing + dc_mid_height + rf_height))]
dc_mid_start_pos = [(leftmost_electrode_x-gnd_width-spacing, (.5*spacing)),
                (leftmost_electrode_x-gnd_width-spacing, -(.5*spacing +dc_mid_height))]

s = trap(spacing,
        gnd_width, gnd_height, gnd_start_pos,
        dc_width, dc_height, dc_start_pos,
        rf_width, rf_height, rf_start_pos,
        dc_mid_width, dc_mid_height, dc_mid_start_pos,
        n, r=2)

[('2', [[(-562.5, 100.0), (-437.5, 100.0), (-437.5, 1105.0), (-562.5, 1105.0)]]), ('3', [[(-437.5, 100.0), (-312.5, 100.0), (-312.5, 1105.0), (-437.5, 1105.0)]]), ('4', [[(-312.5, 100.0), (-187.5, 100.0), (-187.5, 1105.0), (-312.5, 1105.0)]]), ('5', [[(-187.5, 100.0), (-62.5, 100.0), (-62.5, 1105.0), (-187.5, 1105.0)]]), ('6', [[(-62.5, 100.0), (62.5, 100.0), (62.5, 1105.0), (-62.5, 1105.0)]]), ('7', [[(62.5, 100.0), (187.5, 100.0), (187.5, 1105.0), (62.5, 1105.0)]]), ('8', [[(187.5, 100.0), (312.5, 100.0), (312.5, 1105.0), (187.5, 1105.0)]]), ('9', [[(312.5, 100.0), (437.5, 100.0), (437.5, 1105.0), (312.5, 1105.0)]]), ('10', [[(437.5, 100.0), (562.5, 100.0), (562.5, 1105.0), (437.5, 1105.0)]]), ('12', [[(-562.5, -1105.0), (-437.5, -1105.0), (-437.5, -100.0), (-562.5, -100.0)]]), ('13', [[(-437.5, -1105.0), (-312.5, -1105.0), (-312.5, -100.0), (-437.5, -100.0)]]), ('14', [[(-312.5, -1105.0), (-187.5, -1105.0), (-187.5, -100.0), (-312.5, -100.0)]]), ('15', [[(-187.5, -1105.0), (-62.5, -